In [9]:
    import sklearn
    print(sklearn.__version__)

ModuleNotFoundError: No module named 'sklearn'

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier, plot_importance
import shap
import matplotlib.pyplot as plt



ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Load data
df = pd.read_csv("churn_data.csv")

# Data cleaning
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

# Encode target
df['Churn'] = df['Churn'].map({'Yes':1, 'No':0})

# Features & target
X = df.drop(['customerID','Churn'], axis=1)
y = df['Churn']

# Encode categorical variables
X = pd.get_dummies(X, drop_first=True)



In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)



In [ ]:
# Predictions
y_pred = xgb.predict(X_test)

# Evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))



In [ ]:
# Feature importance (global reasons)
plt.figure(figsize=(10,6))
plot_importance(xgb, max_num_features=10)
plt.show()

# SHAP for explainability
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(X_test)

# Global explanation
shap.summary_plot(shap_values, X_test)

# Local explanation for one customer
customer_index = 5  # pick any test customer
shap.force_plot(explainer.expected_value, shap_values[customer_index], X_test.iloc[customer_index])
